# Analyse der Reden von Bundestagsabgeordneten
## 1. Dataset preparation

In [1]:
import pickle, gensim, numpy as np
import pandas as pd 

PICKLE_FOLDER_PATH = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/pickle_TUWien_plenar/'
df = pickle.load(open("/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/pickle_plenar/df_comment_all.pickle_date", "rb" ))

import re

df['Fraktion'] = df['Fraktion'].replace(to_replace=r'(BÜNDNIS(SES)*\s90\/DIE\sGRÜNEN)[$:,]*(?i)', value='BÜNDNIS_90/DIE_GRÜNEN', regex=True)
df['Fraktion'] = df['Fraktion'].replace(to_replace=r'((DIE )*LINKE)[N$:,]*', value='DIE_LINKE', regex=True)
df['Fraktion'] = df['Fraktion'].replace(to_replace=r'(Fraktionslos(?i))', value='Fraktionslos', regex=True)


# Bereits analysierte Reden werden nicht erneut durchlaufen - lese Liste der pickle Files:
import csv

with open(PICKLE_FOLDER_PATH+'plenar_pickle.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        processed_IDs = row

#### Bearbeite/ Vereinheitliche Rede Text

In [2]:
import pandas as pd
import numpy as np
import pymongo
import datetime
import re

pd.set_option('display.max_colwidth', 0)

#time_sample_df = df.loc[(df['Datum'] >= datetime.datetime(2020,1,1,0,0,0)) &
#                        (df['Datum'] <= datetime.datetime(2020,1,30,0,0,0))]


#sample_df = time_sample_df
sample_df = df[df['Rede_Text'].isna() != True]

# Remove punctuation
sample_df['full_text_processed'] = sample_df['Rede_Text'].map(lambda x: re.sub('§[\n\t,\.!?#@\\n"“„\:;&\(\)]', '', x))

# # Convert the titles to lowercase
sample_df['full_text_processed'] = sample_df['full_text_processed'].map(lambda x: x.lower())

sample_df = sample_df[sample_df['full_text_processed'] != '']
sample_df = sample_df[sample_df['full_text_processed'] != ' ']
sample_df = sample_df[sample_df['full_text_processed'] != '   ']

# Print out the first rows of papers
# sample_df['full_text_processed'].head(10)

/usr/local/anaconda3/bin/ipython:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/anaconda3/bin/ipython:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Reden zusammenfügen

In [3]:
# rede_ID = set(df['Rede_ID'])
df_Reden = sample_df[['Rede_ID', 'full_text_processed']]
df_Reden = df_Reden.groupby('Rede_ID')['full_text_processed'].apply(' '.join).reset_index()
df_Reden.index = df_Reden['Rede_ID']

In [4]:
satz = df_Reden["full_text_processed"].str.split('.').apply(pd.Series,1).stack()
satz.index = satz.index.droplevel(-1) # to line up with df's index
satz.name = 'full_text_processed' # needs a name to join

# There are blank or emplty cell values after above process. Removing them
satz.replace('', np.nan, inplace=True)
satz.dropna(inplace=True)

# Features
Code von TUWien angepasst für die Anwendung auf eigene Tweets

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

import pickle, gensim, numpy as np

from utilities import get_train_data, get_test_data, Tokenizer, find_subtoken

PICKLE_FOLDER_PATH = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/pickle_TUWien_plenar'

TRAIN_FILENAME = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/germeval2018.training_all.txt'
#mdp = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/mdp_tweets.txt'

#------------------------------
#source:
#http://www.cl.uni-heidelberg.de/english/research/downloads/resource_pages/GermanTwitterEmbeddings/GermanTwitterEmbeddings_data.shtml
MODEL_FILENAME  = "/home/lisa/Darmstadt/Master Arbeit/06_Analyse/twitter-de_d100_w5_min10.bin" # 821,8 MB
MODEL_DIMENSION = 100

In [85]:
# set(satz.index)
# satz[0:int((0.5*len(satz[satz.index == 'ID1916912700'])))]

In [ ]:
# https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter
from datetime import datetime

for ID in set(satz.index):
    if ID not in processed_IDs:
        print("%s: starting labeling" % (datetime.now().strftime("%H:%M:%S")))
        sample_mdp = np.array(satz[0:int(len(satz[satz.index == ID]))])
        word2vec_model  = gensim.models.KeyedVectors.load_word2vec_format(MODEL_FILENAME, binary=True)

        X_train, y_train_t1, y_train_t2 = get_train_data(TRAIN_FILENAME)
        X_test_mdp                      = sample_mdp

    ### NGRAM FEATURES
    #  * Erstelle n-Gramme mit 3-7 Buchstaben (Funktion: char_vect)
    #  * Erstelle n-Gramme mit 1-3 Wörtern /Funkion: token_vect
    #  * Anwendung auf Training/ Test und mdp Daten

        char_vect  = TfidfVectorizer(analyzer="char", ngram_range=(3, 7), max_df=0.01, min_df=0.0002,
                                     preprocessor=Tokenizer(preserve_case=False, join=True).tokenize)

        token_vect = TfidfVectorizer(analyzer="word", ngram_range=(1, 3), max_df=0.01, min_df=0.0002,
                                     tokenizer=Tokenizer(preserve_case=False, use_stemmer=True).tokenize)

        X_CNGR_train = char_vect.fit_transform(X_train)
        X_CNGR_mdp = char_vect.transform(X_test_mdp)

        X_TNGR_train = token_vect.fit_transform(X_train)
        X_TNGR_mdp  = token_vect.transform(X_test_mdp)

    ### EMB FEATURES
    # * Tweets werden in Token unterteilt
    # * Prüfe ob die Token in einem Token im vortrainierten word2vec Model entsprechen
    # * Wenn nicht, teile Token in Präfix und Suffix und prüfe für diese das word2vec Model (ggf. beide in emb)
    # * emb enthält pro Tweet Vektoren für Token und wird normalisiert mit der Länge des Tweets + ggf extra Tokens
    # * X_EMB enthält die normalisierten Vektoren pro Tweet

        def get_EMB_feats(tweets):   
            tknzr = Tokenizer(preserve_case=True)
            tweets = [tknzr.tokenize(tweet) for tweet in tweets]

            X_EMB = []

            for tweet, i in zip(tweets, range(0,len(tweets))):
                emb = np.zeros(MODEL_DIMENSION)
                extra_tokens = 0

                for token in tweet:
                    try:
                        emb += word2vec_model[token]
                    except:
                        prefix = find_subtoken(token, word2vec_model, mode='initial')
                        suffix = find_subtoken(token, word2vec_model, mode='final')

                        if prefix != None and suffix != None:
                            emb += word2vec_model[prefix] + word2vec_model[suffix]
                            extra_tokens += 1
                        elif prefix != None and suffix == None:
                            emb += word2vec_model[prefix]
                        elif prefix == None and suffix != None:
                            emb += word2vec_model[suffix]           
                emb /= (len(tweet) + extra_tokens)
        #         print(i)
        #         print(len(tweet))
                X_EMB.append(emb)

            return normalize(X_EMB)

        X_EMB_train = get_EMB_feats(X_train)
        X_EMB_mdp  = get_EMB_feats(X_test_mdp)

    # ### TIMP FEATURES
    # * Finden der wichtigen Tokens - also derer die in Tweets der angegebenen Kategorie verwendet werden
    # * Für diese wichtigsten Tokens werden die Features analog der EMB Features aus dem word2vec model abgeleitet
    # * Außerdem werden für alle Tweets analog der EMB feats abgeleitet
    # * Vergleiche mit der Cosine Similarity und gebe die höchsten und niedrigsten Werte pro Tweet zurück

        def k_most_imp_tokenlvl(k, category, max_df=0.01, min_df=0.0002):      
            token_vect = TfidfVectorizer(analyzer="word", ngram_range=(1, 1), lowercase=False,
                                         max_df=max_df, min_df=min_df,
                                         tokenizer=Tokenizer(preserve_case=True).tokenize)

            tfidf = token_vect.fit_transform(X_train)

            vocab = token_vect.vocabulary_
            inv_vocab = {index: word for word, index in vocab.items()}

            if category in ['OTHER', 'OFFENSE']:
                cat_ids = np.where(y_train_t1 == category)
            elif category in ['PROFANITY', 'ABUSE', 'INSULT']:
                cat_ids = np.where(y_train_t2 == category)

            most_imp_ids = np.argsort(np.asarray(np.mean(tfidf[cat_ids], axis=0)).flatten())[::-1]

            most_imp = []
            for index in most_imp_ids:
                most_imp.append(inv_vocab[index])

            return most_imp[:k]

        def get_TIMP_feats(tweets, k, category, max_df=0.01, min_df=0.0002):
            feats_max = []
            feats_min = []

            imp_tokens_vectors = []
            for imp_token in k_most_imp_tokenlvl(k, category, max_df=max_df, min_df=min_df):
                try:
                    imp_tokens_vectors.append(word2vec_model[imp_token])
                except:
                    imp_tokens_vectors.append(np.zeros(MODEL_DIMENSION))

            tknzr = Tokenizer(preserve_case=True)
            tweets = [tknzr.tokenize(tweet) for tweet in tweets]

            for tweet in tweets:
                tweet_vectors = []
                for token in tweet:
                    try:
                        tweet_vectors.append(word2vec_model[token])
                    except:
                        prefix = find_subtoken(token, word2vec_model, mode='initial')
                        suffix = find_subtoken(token, word2vec_model, mode='final')

                        if prefix != None and suffix != None:
                            tweet_vectors.append(word2vec_model[prefix])
                            tweet_vectors.append(word2vec_model[suffix])
                        elif prefix != None and suffix == None:
                            tweet_vectors.append(word2vec_model[prefix])
                        elif prefix == None and suffix != None:
                            tweet_vectors.append(word2vec_model[suffix])
                        else:
                            tweet_vectors.append(np.zeros(MODEL_DIMENSION))

                similarity = cosine_similarity(np.asarray(tweet_vectors), np.asarray(imp_tokens_vectors))

                feats_max.append(np.amax(similarity, axis=0))
                feats_min.append(np.amin(similarity, axis=0))

            return np.concatenate((feats_max, feats_min), axis=1)
        N_TIMP_TASK1 = 1250
        N_TIMP_TASK2 = 170

        X_TIMP_task1_train = \
        np.concatenate((get_TIMP_feats(X_train, N_TIMP_TASK1, 'OTHER'),
                        get_TIMP_feats(X_train, N_TIMP_TASK1, 'OFFENSE')), axis=1)


        X_TIMP_task1_mdp = \
        np.concatenate((get_TIMP_feats(X_test_mdp,  N_TIMP_TASK1, 'OTHER'),
                        get_TIMP_feats(X_test_mdp,  N_TIMP_TASK1, 'OFFENSE')), axis=1)

    ##### CIMP Features

        def k_most_imp_charlvl(k, category, max_df=0.01, min_df=0.0002):    
            char_vect  = TfidfVectorizer(analyzer="char", ngram_range=(3, 7), lowercase=False,
                                         max_df=max_df, min_df=min_df,
                                         preprocessor=Tokenizer(preserve_case=True, join=True).tokenize)

            tfidf = char_vect.fit_transform(X_train)

            vocab = char_vect.vocabulary_
            inv_vocab = {index: word for word, index in vocab.items()}

            if category in ['OTHER', 'OFFENSE']:
                cat_ids = np.where(y_train_t1 == category)
            elif category in ['PROFANITY', 'ABUSE', 'INSULT']:
                cat_ids = np.where(y_train_t2 == category)       

            most_imp_ids = np.argsort(np.asarray(np.mean(tfidf[cat_ids], axis=0)).flatten())[::-1]

            most_imp = []
            for index in most_imp_ids:
                most_imp.append(inv_vocab[index])

            return most_imp[:k]

        def get_CIMP_feats(tweets, k, category, max_df=0.01, min_df=0.0002):
            feats = np.zeros((len(tweets), k))
            for imp_ngram_index, imp_ngram in enumerate(k_most_imp_charlvl(k, category, max_df=max_df, min_df=min_df)):
                for tweet_index, tweet in enumerate(tweets):
                    if tweet.find(imp_ngram) != -1:
                        feats[tweet_index][imp_ngram_index] = 1
            return feats

        N_CIMP_TASK1 = 3200
        N_CIMP_TASK2 = 370

        X_CIMP_task1_train = \
        np.concatenate((get_CIMP_feats(X_train, N_CIMP_TASK1, 'OTHER'),
                        get_CIMP_feats(X_train, N_CIMP_TASK1, 'OFFENSE')), axis=1)


        X_CIMP_task1_mdp = \
        np.concatenate((get_CIMP_feats(X_test_mdp,  N_CIMP_TASK1, 'OTHER'),
                        get_CIMP_feats(X_test_mdp,  N_CIMP_TASK1, 'OFFENSE')), axis=1)

    #### Predictions
        _, y1, y2 = get_train_data(TRAIN_FILENAME)


    ## Funktion für das Aufteilen in Train und Test Sample 
    # -> StratifiedKFold sorgt dafür, dass das prozentuale Verhältnis der Klassen im jeweiligen Sample (Test, Train) gleich ist


        from sklearn.ensemble import ExtraTreesClassifier
        from sklearn.linear_model import LogisticRegression

        from sklearn.model_selection import cross_val_predict, StratifiedKFold

        def get_META_feats(clf, X_train, mdp, y, seeds=[42]):
            feats_train = []
            for seed in seeds:
                skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
                feats_train.append(cross_val_predict(clf, X_train, y=y, method='predict_proba', cv=skf, n_jobs=-1))
            feats_train = np.mean(feats_train, axis=0)
            #print(len(feats_train))
            #print(clf)
            clf.fit(X_train, y)
            feats_mdp = clf.predict_proba(mdp)
            #print(len(feats_mdp))

            return feats_train, feats_mdp

    # ## TASK 1 - Base level predictions
    # Die drei verschiedenen Classifier (clfs_task1) werden auf die Feature Vectoren (base_feats_task1) angewandt.
    # Von einer 10-fold CrossVal wird für den Trainings Feature Satz der Durchschnitt genommen (jeder Spalte).
    # Bei den Test-/mdp Daten wird keine Cross Val durchgeführt (keine y Variablen) sondern nur mit jedem Classifier eine prediction anhand der Feature Vektoren gemacht


        clfs_task1 = [LogisticRegression(class_weight='balanced', max_iter=1000),
                      ExtraTreesClassifier(n_estimators=100, criterion='entropy', n_jobs=-1),
                      ExtraTreesClassifier(n_estimators=100, criterion='gini', n_jobs=-1)]

        base_feats_task1 = [(X_CIMP_task1_train, X_CIMP_task1_mdp),
                            (X_TIMP_task1_train, X_TIMP_task1_mdp),
                            (X_CNGR_train, X_CNGR_mdp),
                            (X_TNGR_train, X_TNGR_mdp),
                            (X_EMB_train, X_EMB_mdp)]
        X_META_task1_train = []
        #X_META_task1_test  = []
        X_META_task1_mdp  = []
        for X_train, mdp in base_feats_task1:                 # X-train z.B X_CIMP_task1_train, mdp z.B X_CIMP_task1_mdp
            for clf in clfs_task1:
                feats = get_META_feats(clf, X_train, mdp, y1)

                X_META_task1_train.append(feats[0])           # aus "get_META_feats: feats_train
                X_META_task1_mdp.append(feats[1])             # aus "get_META_feats: feats_mdp

        X_META_task1_train = np.concatenate(X_META_task1_train, axis=1)
        X_META_task1_mdp  = np.concatenate(X_META_task1_mdp, axis=1)

        clf_task1 = LogisticRegression(C=0.17, class_weight='balanced', max_iter=1000)
        clf_task1.fit(X_META_task1_train, y1) 

        preds_task1 = clf_task1.predict(X_META_task1_mdp) 

        pd.set_option('display.max_colwidth', 0)
        satz['predict'] = preds_task1
        print(ID)
        print(satz['predict'])
        print("%s: end labeling" % (datetime.now().strftime("%H:%M:%S")))
        
    else:
        print("ID: " + ID + " wurde bereits verwendet")


07:26:20: starting labeling


/usr/local/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This ca

ID1911500400
['OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER'
 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER'
 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER'
 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER'
 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER'
 'OFFENSE']
08:36:55: end labeling
08:36:55: starting labeling


/usr/local/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


ID192611800
['OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER' 'OTHER']
09:47:07: end labeling
09:47:07: starting labeling


# Darstellung der Ergebnisse
### Laden und zusammenfügen der DataFrames

In [70]:
import pickle
import pandas as pd
# Bereits analysierte Reden werden nicht erneut durchlaufen - lese Liste der pickle Files:
import csv

with open(PICKLE_FOLDER_PATH+'pickle_Reden.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        processed_IDs = row
        
with open(PICKLE_FOLDER_PATH+'pickle_IDs.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        IDs = row
        
PicklePath = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/pickle_TUWien_plenar/'

dic_reden = {}
i = 0
for file, ID in zip(processed_IDs, IDs):
    #print(ID)
    df_file = pickle.load(open(PicklePath+file, "rb" ))         # Öffne file (enthält schlecht strukturierte Daten)
    df = df_file[df_file.index == ID]                           # Einschränken auf aktuelle ID weil df alle IDs enthält aber nur predict für genannte!
    pred_array = list(df_file[df_file.index == 'predict'])[0]   # Speichere predictions (stehen am Ende des files) in eigenem array
    df = pd.DataFrame(df)
    df['predict'] = pred_array                                  # Füge prediction array als Spalte an df an
    dic_reden[ID] = df
    
reden_df = pd.concat(dic_reden, axis = 0)
reden_df.index = reden_df.index.get_level_values(1)



In [79]:
import pickle, gensim, numpy as np
import pandas as pd 

PICKLE_ERSTER_DURCHLAUF = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/Plenar_TUWien_ersterDurchlauf/'
#pickle.dump(reden_df, open(PICKLE_ERSTER_DURCHLAUF + 'reden_sent_TUWien_all.p', "wb" ))
reden_df = pickle.load(open(PICKLE_ERSTER_DURCHLAUF + 'reden_sent_TUWien_all.p', "rb" ))

In [ ]:
## ToDo:
## - Erstelle df mit Labeln aus compare folder -> Durchläufe nur mit halbem trainingsset
## - Überarbeite LabelAlgo - damit nicht als chaos-df gespeichert
## - Lasse mit neuem/final Trainingssett durchlaufen 

In [84]:
import pickle
import pandas as pd
# Bereits analysierte Reden werden nicht erneut durchlaufen - lese Liste der pickle Files:
import csv

PicklePath = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/Plenar_TUWien_ersterDurchlauf/compare_half_trainSet/'

with open('/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/Plenar_TUWien_ersterDurchlauf/compare_half_trainSet/pickle_rede_comp.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        processed_IDs = row
        
#with open(PicklePath+'pickle_rede_comp_IDs.txt') as csv_file:
with open('/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/Plenar_TUWien_ersterDurchlauf/compare_half_trainSet/pickle_rede_comp_IDs.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        IDs = row
        

dic_reden = {}
i = 0
for file, ID in zip(processed_IDs, IDs):
    #print(ID)
    df_file = pickle.load(open(PicklePath+file, "rb" ))         # Öffne file (enthält schlecht strukturierte Daten)
    df = df_file[df_file.index == ID]                           # Einschränken auf aktuelle ID weil df alle IDs enthält aber nur predict für genannte!
    pred_array = list(df_file[df_file.index == 'predict'])[0]   # Speichere predictions (stehen am Ende des files) in eigenem array
    df = pd.DataFrame(df)
    df['predict'] = pred_array                                  # Füge prediction array als Spalte an df an
    dic_reden[ID] = df
    
reden_df_comp = pd.concat(dic_reden, axis = 0)
reden_df_comp.index = reden_df_comp.index.get_level_values(1)

In [87]:
reden_df_comp.loc[reden_df_comp['predict'] == 'OFFENSE'] 
# Bei Training mit halbem Trainingssatz ist kein Satz als OFFENSE gelabelt worden!

,full_text_processed,predict
Rede_ID,,


In [89]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', 0)
reden_df.loc[reden_df['predict'] == 'OFFENSE']

,full_text_processed,predict
Rede_ID,,
ID1910106500,diese regierung und diese eu bewirken das gegenteil davon,OFFENSE
ID1910402400,"wir sind nicht der auffassung, dass beide das alleine können",OFFENSE
ID191200200,"es ist wichtig, gleichzeitig den haushalt weiter zu konsolidieren",OFFENSE
ID1912502900,"es geht darum, dass wir die inländische wertschöpfung in den nächsten jahren steigern, dass wir die lebensqualität der deutschen bzw",OFFENSE
ID1913000600,"gerade deshalb erfordert es ein neues denken, einen neuen politikansatz, der eben nicht die vermeintlichen gegensätze sucht, der nicht den kleinsten gemeinsamen nenner findet",OFFENSE
ID1913004600,"ich finde, parlamentarische kontrolle gehört ins parlament, und am ende des tages werden auch nicht alle fraktionen im aufsichtsrat vertreten sein können",OFFENSE
ID1913412100,ich will ihnen gerne auch noch mal zurufen: schließen sie sich der hohen geschwindigkeit der koalition an! sie können es noch schaffen,OFFENSE
ID191401500,"deshalb die lehre aus dem desaster in syrien: macht europa stark! lasst uns im europäischen verbund agieren! dann können wir uns in dieser region engagieren, und dann können wir dafür sorgen, dass das leid geringer wird",OFFENSE
ID1915206400,"– dass möglicherweise anträge abgelehnt werden, gibt es auch im sozialrecht",OFFENSE


In [ ]:
import pickle, gensim, numpy as np
import pandas as pd 

pickle.dump(reden_df_comp, open(PicklePath + 'comp_reden_sent_TUWien_all.p', "wb" ))
#reden_df_comp = pickle.load(open(PicklePath + 'comp_ reden_sent_TUWien_all.p', "rb" ))